<a href="https://colab.research.google.com/github/offmann/demande_des_valeurs_foncieres/blob/master/DVF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>**Récupération des données de 2014 à 2018**

In [0]:
# Imports 

import pandas as pd
from google.colab import files

In [0]:
%%shell

mkdir -p data
cd data
# Téléchargement des fichiers DVF
for ANNEE in $(seq 2014 2018)
do
  wget -nc http://data.cquest.org/dgfip_dvf/valeursfoncieres-$ANNEE.txt.gz
  wget -nc https://cadastre.data.gouv.fr/data/hackathon-dgfip-dvf/contrib/etalab-csv/$ANNEE/full.csv.gz -O $ANNEE-full.csv.gz
done

--2019-07-01 08:13:20--  http://data.cquest.org/dgfip_dvf/valeursfoncieres-2014.txt.gz
Resolving data.cquest.org (data.cquest.org)... 163.172.103.163
Connecting to data.cquest.org (data.cquest.org)|163.172.103.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49290306 (47M) [application/octet-stream]
Saving to: ‘valeursfoncieres-2014.txt.gz’

valeursfoncieres-20 100%[===================>]  47.01M  14.0MB/s    in 3.4s    

2019-07-01 08:13:24 (14.0 MB/s) - ‘valeursfoncieres-2014.txt.gz’ saved [49290306/49290306]

--2019-07-01 08:13:24--  https://cadastre.data.gouv.fr/data/hackathon-dgfip-dvf/contrib/etalab-csv/2014/full.csv.gz
Resolving cadastre.data.gouv.fr (cadastre.data.gouv.fr)... 54.36.212.78
Connecting to cadastre.data.gouv.fr (cadastre.data.gouv.fr)|54.36.212.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71732630 (68M) [application/octet-stream]
Saving to: ‘2014-full.csv.gz’

2014-full.csv.gz    100%[===================>] 

In [0]:
! ls data/

2014-full.csv.gz  2018-full.csv.gz		valeursfoncieres-2017.txt.gz
2015-full.csv.gz  valeursfoncieres-2014.txt.gz	valeursfoncieres-2018.txt.gz
2016-full.csv.gz  valeursfoncieres-2015.txt.gz
2017-full.csv.gz  valeursfoncieres-2016.txt.gz


In [0]:
%%shell

for ANNEE in $(seq 2014 2018)

do
  gunzip data/valeursfoncieres-$ANNEE.txt.gz
done  

In [0]:
! ls data/

2014-full.csv.gz  2018-full.csv.gz	     valeursfoncieres-2017.txt
2015-full.csv.gz  valeursfoncieres-2014.txt  valeursfoncieres-2018.txt
2016-full.csv.gz  valeursfoncieres-2015.txt
2017-full.csv.gz  valeursfoncieres-2016.txt


In [0]:
def dvf (year):
  
  # Read data
  data = pd.read_csv('data/valeursfoncieres-' +str(year)+ '.txt', sep = '|', header = None, low_memory=False)
  data.columns = data.iloc[0]
  data = data.reindex(data.index.drop(0))
  
  
  # Remove rows containing NAs values among the subset of columns 
  data = data.dropna(subset=['Valeur fonciere','No voie','Type de voie','Voie'])

  
  # Filter on departements ( 5 Départements of North of France)
  departement_condition = (data['Code departement']=='59') | (data['Code departement']=='80') | (data['Code departement']=='02') | (data['Code departement']=='08') | (data['Code departement']=='62')
  data = data[departement_condition]

  
  # Remove unnecessary columns 
  cols = [0,1,2,3,4,5,6,22,23,24,25,26,27,28,29,30,31,32,33,35,37,40,41]
  data.drop(data.columns[cols],axis=1,inplace=True)


  # Format No Voie column 
  data['No voie'] = data['No voie'].astype(str)
  data['No voie'] = data['No voie'].str.split('.').str[0]
  data.head()


  # Create Adresse variable
  col_to_concat = [3,4,6]
  data['adresse']= data['No voie'] + ' ' + data['Type de voie'] + ' ' + data['Voie'] + ' ' + data['Code postal'] + ' ' + data['Commune']


  # Export CSV file
  data.to_csv('data_fonciere_'+str(year)+'.csv', encoding='utf-8')
  files.download('data_fonciere_'+str(year)+'.csv')
  
  
  # return head of data frame
  
  return data.sort_values(by=['Date mutation'], ascending=False).head(), data.shape, data

In [0]:
# Utilisation pour l'année 2018
head_2018, shape_2018, dataset_2018 = dvf(2018)